### Préparation du jeu de données

L'exécution de ce notebook est parfaitement optionnelle, les données étant fournies préparées.<br/>
On le met à disposition pour la simple curiosité des étudiant·e·s.

#### Préparation manuelle

On commence par télécharger les données [ici](http://ai.stanford.edu/%7Eamaas/data/sentiment/) (Large Movie Review Dataset v1.0) et par décompresser l'archive (avec tar soux Linux, ou 7Zip sous Windows) afin d'en extraire le contenu dans un dossier donné.

#### Lecture des données

On va lire les données pour constituer un unique DataFrame contenant le corpus d'entraînement, et un second pour le test.

Dans un premier temps on définit les fonctions nécessaires à lire un fichier, puis à charger tout un sous-corpus :

In [ ]:
import os  # standard library module
import re  # standard library module

import pandas as pd  # third-party module (`pip install pandas`)


def read_rating_file(path):
    """Read a given movie review file.
    
    Return both the review itself (str) and the associated rate (int).
    """
    # Parse file name to get the review's rate (format: 'id_rate.txt').
    name = os.path.basename(path)[:-4]
    rate = int(name.rsplit('_', 1)[1])
    # Read the file's content, remove trailing whitespaces.
    with open(path, mode='r', encoding='utf-8') as file:
        text = file.read().strip('\n')
    # Run minimal normalization on separation characters.
    text = re.sub('<br ?/>', ' ', text)  # remove html linebreaks
    text = re.sub(r'[\n\t]', ' ', text)  # remove linebreaks and tabs
    text = re.sub(r'\s\s+', ' ', text)   # remove redundant whitespaces
    # Return the text and the rate.
    return (text, rate)


def load_imdb_subset(path):
    """Load all samples from either the train of test IMDB review dataset.
    
    path : path to either the 'train' or 'test' folder in the IMDB dataset
           initial uncompressed export
           
    Return a pd.DataFrame storing reviews, with both their 'text', 'rate'
    and 'polarity'.
    """
    samples = []
    for folder in ('neg', 'pos'):
        folder = os.path.join(path, folder)
        for file in sorted(os.listdir(folder)):
            text, rate = read_rating_file(os.path.join(folder, file))
            samples.append({'text': text, 'rate': rate, 'polarity': int(rate >= 7)})
    return pd.DataFrame(samples)

Ensuite, on utilise ces fonctions pour charger les deux DataFrames pandas.
On en profite pour visualiser les premières lignes du jeu "train" et les dimensions des deux jeux (250k reviews par jeu, avec une égalité parfaite positif/négatif).

In [ ]:
# Load the two subdatasets (this can take a bit of time).
train = load_imdb_subset('./imdb/train')
test = load_imdb_subset('./imdb/test')

In [ ]:
# Show the first lines of the train set (display cut text).
train.head()

In [ ]:
# Display the shapes of the DataFrames.
print(train.shape)
print(test.shape)

In [ ]:
# Display the share of positive (>= 7) reviews in each subset.
print(train['polarity'].mean())
print(test['polarity'].mean())

In [ ]:
# Display the number of reviews per exact rate in the train set.
print(train['rate'].value_counts().sort_index())

#### Préparation des données

On pourraît s'arrêter là, mais on va plutôt :
* faire un petit peu de normalisation en plus pour se faciliter la vie ensuite

In [ ]:
# Stack both subsets into a single DataFrame.
data = pd.concat([train, test], axis=0)
data.index = list(range(len(data)))
print(data.shape)

In [ ]:
# Let's remove non-informative unicode characters (things such as '\x8d').
import unicodedata  # third-party module (`pip install unicodedata`)

def remove_noncharacters(text):
    """Remove unicode 'other' characters from a text."""
    return ''.join(
        char for char in unicodedata.normalize('NFD', text)
        if not unicodedata.category(char).startswith('C')
    )

data['text'] = data['text'].apply(remove_noncharacters)

_Note: On garde les accents, qui semblent utilisés avec parcimonie (noms propres et emprunts au Français, e.g. "cliché") et donc convoyer une information pertinente. De même, on laisse à ce stade toute la ponctuation, y compris des symboles qui n'ont probablement aucun sens pris seuls (e.g. pas dans un n-gram ou dans un modèle faisant un usage séquentiel du texte)._

* produire un nouveau split train / validation / test avec des proportions 70% / 15% / 15%

In [ ]:
import numpy as np  # third-party library, dependency of pandas

# Set up a Random Number Generator; use a seed for reproducibility.
rng = np.random.Generator(np.random.MT19937(seed=0))
# Randomly split the dataset in disjoint subsets.
idx = data.index.values  # copy the dataset's index
rng.shuffle(idx)         # randomly shuffle this copy
train = data.loc[idx[:35000]]       # 70% of samples
valid = data.loc[idx[35000:42500]]  # 15% of samples
test = data.loc[idx[42500:]]        # 15% of samples

#### Export des données

Finalement, on exporte les trois jeux de données au format tsv (tab-separated value).

_Note: on utilise la tabulation plutôt que la virgule (standard csv, comma-separated value) car les textes contiennent des virgules ; on s'évite ainsi d'éventuels problèmes de parseur._

In [ ]:
# Build the output data folder if needed.
if not os.path.isdir('./data'):
    os.mkdir('./data')
# Write down tsv files.
train.to_csv('./data/train.tsv', sep='\t', index=False)
valid.to_csv('./data/valid.tsv', sep='\t', index=False)
test.to_csv('./data/test.tsv', sep='\t', index=False)